**Building a Basic RAG Tutorial Using Gemini 1.5 Pro**

First you need to import the libraries:
-google.generativeai for the Gemini model
-SentenceTransformer for creating embeddings
-cosine_similarty for measuring the similarity between two texts by calculating the angle between their vector representations.
-numpy for handling data

In [ ]:
import google.generativeai as genai
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Now you can set up your Gemini API key with your own key

In [ ]:
# Set up Gemini API (replace with your actual API key)
genai.configure(api_key="ghsfjsfjrsrtyey34343jfh")

Then, we initialise:

*  LLM → Gemini model that I am using Gemini 1.5 pro but feel free to replace it
*  With any other model you prefer such as Gemini 1.5 flash.
*  Embeddings: We’ll create embeddings for our sample knowledge base.
*  Knowlege Base: a simple list of facts about Paris.

In [ ]:
# Initialize the model
model = genai.GenerativeModel('gemini-1.5-pro')

# Initialize sentence transformer for embedding
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Sample knowledge base
knowledge_base = [
    "The capital of France is Paris.",
    "The Eiffel Tower is located in Paris.",
    "The Louvre Museum houses the Mona Lisa painting.",
    "French cuisine is known for dishes like croissants and baguettes.",
    "The French Revolution began in 1789.",
    "Paris is divided into 20 arrondissements or districts.",
    "The Seine River flows through the heart of Paris.",
    "Notre-Dame Cathedral is a famous Parisian landmark.",
    "Paris is often referred to as the 'City of Light'.",
    "The Paris Metro is one of the busiest subway systems in Europe."
]

Defining functions:
*   retrieve_context: finds the most pertinent data from the knowledge base based on the query using cosine similarity.
*   generate_response: uses Gemini to create an answer using the retrieved information and the Gemini LLM.
*   rag_with_gemini: brings together the retrieval and generation processes to produce a final response.

In [ ]:
# Function to retrieve relevant context
def retrieve_context(query, knowledge_base, top_k=2):
    query_embedding = embedder.encode([query])
    context_embeddings = embedder.encode(knowledge_base)

    similarities = cosine_similarity(query_embedding, context_embeddings)[0]
    top_indices = np.argsort(similarities)[-top_k:]

    return [knowledge_base[i] for i in reversed(top_indices)]

# Function to generate response using Gemini
def generate_response(query, context):
    prompt = f"Context: {' '.join(context)}\n\nQuestion: {query}\n\nAnswer:"
    response = model.generate_content(prompt)
    return response.text

# Main RAG function
def rag_with_gemini(query):
    context = retrieve_context(query, knowledge_base)
    response = generate_response(query, context)
    return response

To test the model, let’s test our RAG system with a simple query: ‘What can you tell me about Paris?’ The system effectively retrieved relevant information from our basic knowledge base about Paris and generates a comprehensive response.

In [ ]:
# Example usage
query = "What can you tell me about Paris?"
answer = rag_with_gemini(query)
print(f"Question: {query}")
print(f"Answer: {answer}")

Question: What can you tell me about Paris?
Answer: Paris, the capital of France, is often called the "City of Light". This nickname is believed to have originated either from its fame as a center of education and ideas during the Enlightenment, or its early adoption of gas street lighting. 

Beyond its nickname, Paris is known for:

* **Iconic landmarks:** The Eiffel Tower, Louvre Museum (home to the Mona Lisa), Arc de Triomphe, Notre Dame Cathedral, and Sacré-Cœur Basilica are just a few of its famous attractions.
* **Romance and beauty:**  Paris is considered one of the most romantic cities in the world, known for its charming streets, bridges over the Seine River, and cozy cafes.
* **Fashion and culture:**  Paris is a global center for fashion, art, and cuisine, boasting world-renowned designers, museums, and restaurants. 
* **History:** From its Roman roots to the French Revolution, Paris has a rich and fascinating history that is visible throughout the city. 

This is just a glim